# pixel_acc
- 总类别和正确的类别之比
- 预测时挑出结果中概率最大的类别 使用索引进行运算

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## torch.max 实例用法
torch.max 会返回 torch.return_types.max 的实例 这个实例是一个二元矩阵 其一包含具体的值 其二包含值中的索引
在pixel_acc中我们只需要他的索引
##
## 类型转换小tips
创建变量时需要规定变量类型 在使用时按需要进行转换

In [2]:
a = torch.randn(4, 4)
print(a)
b = torch.max(a, dim=1)
print('finish a ')
print(b)

tensor([[ 0.6226,  2.8619,  0.4566,  0.0505],
        [-0.6126,  0.7075, -0.0189,  0.1663],
        [-0.6320, -1.1780,  1.7127,  1.4621],
        [-0.0197, -0.5695, -0.6422, -0.3411]])
finish a 
torch.return_types.max(
values=tensor([ 2.8619,  0.7075,  1.7127, -0.0197]),
indices=tensor([1, 1, 2, 0]))


In [5]:
def pixel_acc(pred, label, ignore_index=-1):
    # preds 取的是概率最大的类别 所以这里可以使用max
    _, preds = torch.max(pred, dim=1)
    # long() 将输出规定为长整型
    valid = (label != ignore_index).long()
    acc_sum = torch.sum(valid * (preds == label).long())
    pixel_sum = torch.sum(valid)
    acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
    return acc

# part_pixel_acc
- part_pixel_acc 是细分类的小样本的acc

In [ ]:
def part_pixel_acc(pred_part, gt_seg_part, gt_seg_object, object_label, valid):
    mask_object = (gt_seg_object == object_label)
    _, pred = torch.max(pred_part, dim=1)
    acc_sum = mask_object * (pred == gt_seg_part)
    acc_sum = torch.sum(acc_sum.view(acc_sum.size(0), -1), dim=1)
    acc_sum = torch.sum(acc_sum * valid)
    pixel_sum = torch.sum(mask_object.view(mask_object.size(0), -1), dim=1)
    pixel_sum = torch.sum(pixel_sum * valid)
    return acc_sum, pixel_sum

# part_loss

## F.nll_loss 最大似然损失函数
- 最大似然可看作最小化训练集上的经验分布和模型分布的差异，两者之间差异程度可以通过KL散度衡量
- 最小化KL散度其实在最小化分布之间的交叉熵
#
## torch.clamp(input, min, max, out=None)
- 限制输入张量的每个元素范围 返回一个新张量
- 若是小于下限 则用规定下限替换 若是大于上限 用规定上限替换
#
## view()
重新定义矩阵的形状 定义某个参数为-1时 代表动态调整该维度上的元素个数 以保证元素的总和不变
#
## shape[]
读取矩阵长度 shape[0]为读取矩阵第一维度的长度
shape 的输入参数可以是一个整数(表示维度) 也可以是一个矩阵

In [ ]:
def part_loss(pred_part, gt_seg_part, gt_seg_object, object_label, valid):
    mask_object = (gt_seg_object == object_label)
    loss = F.nll_loss()
    loss= loss * mask_object.float()
    loss = torch.sum(loss.view(loss.size(0), -1), dim=1)
    nr_pixel = torch.sum(mask_object.view(mask_object.shape[0], -1), dim=1)
    sum_pixel = (nr_pixel * valid).sum()
    loss = (loss * valid.float()).sum() / torch.clamp(sum_pixel, 1).float()
    return loss

In [3]:
print(a)
b_clamp = torch.clamp(a, 1)
print(b_clamp)

tensor([[ 0.6226,  2.8619,  0.4566,  0.0505],
        [-0.6126,  0.7075, -0.0189,  0.1663],
        [-0.6320, -1.1780,  1.7127,  1.4621],
        [-0.0197, -0.5695, -0.6422, -0.3411]])
tensor([[1.0000, 2.8619, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.7127, 1.4621],
        [1.0000, 1.0000, 1.0000, 1.0000]])


# accuracy(preds, label)


In [ ]:
def accuracy(preds, label):
    valid = (label >= 0)
    acc_sum = (valid * (preds == label)).sum()
    valid_sum = valid.sum()
    acc = float(acc_sum) / (valid_sum + 1e-10)
    return acc, valid_sum